In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from math import exp
import seaborn as sns
import matplotlib

In [ ]:
# function which skecths decision boundaries and margins
def plot_svc_decision_function(model, title = 'Plot', ax=None, plot_support=True):
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    ax.set_title(title)
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
# Load the data
data = pd.read_csv("/kaggle/input/2d-vector-dataset-with-class-number/Coordinates.csv")
data.head()

In [ ]:
# Visualizing the dataset
plt.scatter(data['x1'], data['x2'])
plt.show()

In [ ]:
# Visualizing the dataset with class 
label = data["class"]
colors = ['red','blue']

fig = plt.figure(figsize=(8,4))
plt.scatter(data['x1'], data['x2'], c=label, cmap=matplotlib.colors.ListedColormap(colors))

cb = plt.colorbar()
loc = [1,2]
cb.set_ticks(loc)
cb.set_ticklabels(['class1', 'class2'])

In [ ]:
# separate dataset as ClassC1 and ClassC2
dfClass1 = data[data["class"] == 1]
dfClass2 = data[data["class"] == 2]

In [ ]:
# seperate dataset as train and test
trainC1, testC1, ytrainC1, ytestC1 = train_test_split(dfClass1.iloc[:, :-1], dfClass1.iloc[:, -1], test_size=0.1, random_state=42, shuffle=True)
trainC2, testC2, ytrainC2, ytestC2 = train_test_split(dfClass2.iloc[:, :-1], dfClass2.iloc[:, -1], test_size=0.1, random_state=42, shuffle=True)

In [ ]:
# combine C1 and C2 classes
totalTrain = trainC1.append(trainC2, ignore_index=True)
totalTrainY = ytrainC1.append(ytrainC2, ignore_index=True)

In [ ]:
label = totalTrainY
fig = plt.figure(figsize=(8,4))
plt.scatter(totalTrain['x1'], totalTrain['x2'], c=label, cmap=matplotlib.colors.ListedColormap(colors))
cb = plt.colorbar()
loc = [1,2]
cb.set_ticks(loc)
cb.set_ticklabels(['class1', 'class2'])

In [ ]:
model = SVC(kernel='linear')
model.fit(totalTrain, totalTrainY)

In [ ]:
totalTest = testC1.append(testC2, ignore_index=True)
totalTestY = ytestC1.append(ytestC2, ignore_index=True)

In [ ]:
result_svm = model.predict(totalTest)

In [ ]:
cm = confusion_matrix(totalTestY,result_svm)
print('Accuracy Score: ',accuracy_score(totalTestY,result_svm))
print('Confusion Matrix For Test Set:')
print(cm)
print('-'*55)
print('')
print(classification_report(totalTestY, result_svm))

In [ ]:
plt.figure(figsize=(3,3))
plt.subplots_adjust(wspace = 0.4, hspace= 0.4)
plt.title("Confusion Matrix For Test set")
sns.heatmap(cm,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 16})

In [ ]:
# skecth test data set without seperation
label = totalTestY
fig = plt.figure(figsize=(8,4))
plt.scatter(totalTest['x1'], totalTest['x2'], c=label, cmap=matplotlib.colors.ListedColormap(colors))

cb = plt.colorbar()
loc = [1,2]
cb.set_ticks(loc)
cb.set_ticklabels(['class1', 'class2'])

In [ ]:
# Skecth test dataset with decision boundary(full line), margin(dashed lines).
fig = plt.figure(figsize=(8,4))
plt.scatter(totalTest['x1'], totalTest['x2'], c=totalTestY, s=30, cmap=matplotlib.colors.ListedColormap(colors))

title = 'Dataset after classification for Testset'
plot_svc_decision_function(model, title)
cb = plt.colorbar()
loc = [1,2]
cb.set_ticks(loc)
cb.set_ticklabels(['class1', 'class2'])

In [ ]:
# Skecth train dataset with decision boundary(full line), margin(dashed lines).
fig = plt.figure(figsize=(8,4))
plt.scatter(totalTrain['x1'], totalTrain['x2'], c=totalTrainY, s=30, cmap=matplotlib.colors.ListedColormap(colors))

title = 'Dataset after classification for Trainset'
plot_svc_decision_function(model, title)
cb = plt.colorbar()
loc = [1,2]
cb.set_ticks(loc)
cb.set_ticklabels(['class1', 'class2'])